# A maxima implementation for separation of variables

We discuss a simple but effective implementation of a method to separate variables. Separation of variables is a well-known and much-used technique to solve ordinary and partial differential equations. 

An expression $F(x,y)$ with the 2 independent variables $x$ and $y$ is separable when $\frac{\partial F}{\partial x}/F = \frac{F_x}{F}$ depends only on $x$, see [1], [2]. This is easy to see because if $F$ is separable, it can be written as $F=f(x)\cdot R(y)$. If on both sides we take the derivative with respect to $x$ and divide by $F$ we obtain: $\frac{F_x}{F} = \frac{f_x}{f}$. Since we know that the right-hand side is a function of only x, the left-hand side must be as well.
The separated function $f(x)$ can then be found by solving the differential equation:

$|f(x)| = e^{\int(F_x/Fdx)}$ (1)

The function $g(y)$ can be found in a similar way, e.g. by computing:

$|g(y)| = e^{\int(F_y/Fdy)}$, (2)

or by using $g(y) = F(x,y)/f(x)$. Note that the latter approach might be computationally more costly because we have to properly factor out the x-dependency. With the above aproach eqs. (1)-(2), we lose any constant factors, e.g. when $F(x,y)=C \cdot f(x) \cdot g(y)$, we have lost $C$. We can find $C$ during the evaluation of our results: we check that 
$C = \frac{F(x,y)}{f(x)\cdot g(y)}$ is free of $x$ and $y$.

 
The maxima implementation for separation of variables has been implemented as follows:


# Function and variable index

<u>Function:</u> <b>separable</b>(expr,x,y,[options])

If the input expression is a separable function in the independent variables $x$ and $y$, e.g. the expression can be written as $C\cdot f(x)\cdot g(y)$, with $C$ a constant independent of $x,y$, then separable(expr,x,y) returns a list of the form $[f(x), C\cdot g(y)]$, or false when the expression is not separable.
When the optional input option 'splitConstant=true is set, the constant factor will be separated as well and a list $[f(x),g(y),C]$ will be returned.


<u>Function:</u> <b>isSeparable</b>(expr,x,y)

If the input expression can be written as a separable expression $f(x)*g(y)$, return true, otherwise return false.

<u>Function:</u> <b>constant_factors</b>(expr,varlist)

If the input expression can be written as $C\cdot F(x,y)$, with $C$ an expression free of the variables in the list varlist, constant_factors will return the constant, or 1.


# Example
For some expressions is it not immediately clear from inspection that they are separable. For instance the expression

$e^{x^2+y^2}(\cos(x+y) + \cos(x-y))$ 

can be rewritten as the product of separable functions: 

$e^{x^2} \cdot e^{y^2} \cdot 2\cos(x) \cos(y)$

The maxima implementation of the above algorithm recognizes this without relying on trigonometric identities to recognize this separation.  

In [1]:
kill(all);

done

In [2]:
batch("/home/nijso/mathematics/maxima_files/separable.mac");



read and interpret file: /home/nijso/mathematics/maxima_files/separable.mac
(%i1) put('separable,2,'version)
(%i2) DEBUGFLAG:5
(%i3) MAX_LENGTH_FOR_SEPARABILITY:8000
(%i4) MAX_LENGTH_FOR_SIMPLIFICATION:8000
(%i5) ratfac:true
(%i6) radsubstflag:true
(%i7) dependentform(_expr):=block([_dc:dependencies],
                    for _i thru length(_dc) do
                        _expr:subst(op(_dc[_i]),_dc[_i],_expr),return(_expr))
(%i8) separable(_F,_x,_y,[options]):=block(
                [_F1,_C,_dF,_S,_f:[],_g:[],_P,_Q,_ratvars,_L,_cfp,_cfq,_SP,
                 _SQ,_Res,isSeparable:false],
                splitConstant:assoc('splitConstant,options,false),
                for _i thru length(dependencies) do
                    (_odetmp:errcatch(subst(dependencies[_i],
                                            op(dependencies[_i]),_F)),
                     if _odetmp # [] then _F:_odetmp),_F:flatten([_F])[1],
                dprint(5,"explicit expression =",_F," ",_x," ",_y),
          

"/home/nijso/mathematics/maxima_files/separable.mac"

In [3]:
expr1 : 6*e^(x^2+y^2)*(2*cos(x+y) + 2*cos(x-y));

6*e^(y^2+x^2)*(2*cos(y+x)+2*cos(y-x))

In [4]:
separable(expr1,x,y);

                          2    2
                         y  + x
explicit expression = 6 e        (2 cos(y + x) + 2 cos(y - x))   x   y 
                          2    2
                         y  + x
explicit expression = 6 e        (2 cos(y + x) + 2 cos(y - x))   x   y 
12*e^(y^2+x^2)*(cos(y+x)+cos(y-x))$
F1= done 
we have product terms 
            2    2
           y  + x
L =  [12, e       , cos(y + x) + cos(y - x)] 
          2    2
         y  + x
res =  [e       , cos(y + x) + cos(y - x)] 
   length S =  34 
         2    2
        y  + x
F1= 12 e        (cos(y + x) + cos(y - x)) 
        2    2
       y  + x
P= 12 e        (cos(y + x) + cos(y - x)) 
Q= 1 
            2    2
           x    y
ratvars= [e  , e  , cos(y - x), cos(y + x)] 
(e^((-y^2)-x^2)*(12*e^(y^2+x^2)*(sin(y-x)-sin(y+x))
                +24*e^(y^2+x^2)*log(e)*x*(cos(y+x)+cos(y-x))))
 /(12*(cos(y+x)+cos(y-x)))$
simplify::START, S= done 
dependencies =  [] 
(e^((-y^2)-x^2)*(12*e^(y^2+x^2)*(sin(y-x)-sin(y+x))
    

[e^x^2*cos(x),24*e^y^2*cos(y)]

The constant factor is always added to the second term $g(y)$. We can also separate the constant by using the optional command 'splitConstant=true. The constant factor is then added to the list.

In [5]:
expr2 : separable(expr1,x,y,'splitConstant=true);

                          2    2
                         y  + x
explicit expression = 6 e        (2 cos(y + x) + 2 cos(y - x))   x   y 
                          2    2
                         y  + x
explicit expression = 6 e        (2 cos(y + x) + 2 cos(y - x))   x   y 
12*e^(y^2+x^2)*(cos(y+x)+cos(y-x))$
F1= done 
we have product terms 
            2    2
           y  + x
L =  [12, e       , cos(y + x) + cos(y - x)] 
          2    2
         y  + x
res =  [e       , cos(y + x) + cos(y - x)] 
   length S =  34 
         2    2
        y  + x
F1= 12 e        (cos(y + x) + cos(y - x)) 
        2    2
       y  + x
P= 12 e        (cos(y + x) + cos(y - x)) 
Q= 1 
            2    2
           x    y
ratvars= [e  , e  , cos(y - x), cos(y + x)] 
(e^((-y^2)-x^2)*(12*e^(y^2+x^2)*(sin(y-x)-sin(y+x))
                +24*e^(y^2+x^2)*log(e)*x*(cos(y+x)+cos(y-x))))
 /(12*(cos(y+x)+cos(y-x)))$
simplify::START, S= done 
dependencies =  [] 
(e^((-y^2)-x^2)*(12*e^(y^2+x^2)*(sin(y-x)-sin(y+x))
    

[e^x^2*cos(x),e^y^2*cos(y),24]

We can also call the routine responsible for the separation of the constant factor independently:

In [6]:
constant_factors(5*c*x+5*c,[x]);

5*c

constant_factors also tries to find a shared minus sign. If all terms in an expression of the form a+b+c+d have a minus sign, e.g. expr: -a-b-c-d, then the minus sign will also be factored out.

In [7]:
constant_factors(-5*c*x-5*c,[x]);

-5*c

Sometimes we just want to know if an expression is separable or not. We can then skip some internal computations.

In [8]:
isSeparable(x*y,x,y);

1/x$
simplify::START, S= done 
dependencies =  [] 
1/x$
simplify:: S= done 
   length S =  3 
ratvars =  [x] 
   N,Nnew =  3   3 
1/x$
2. S= done 
      1
3. S= - 
      x
1/x$
simplify::END, S= done 
f is separable 


true

# Separation of variables for an ODE

We can use separation of variables to solve ordinary differential equations. For the general separable ode:

In [9]:
ode:'diff(y,x)=f(x)*g(y);

'diff(y,x,1) = f(x)*g(y)

In [10]:
S:separable(rhs(ode),x,y);

explicit expression = f(x) g(y)   x   y 
explicit expression = f(x) g(y)   x   y 
f(x)*g(y)$
F1= done 
we have product terms 
L =  [f(x), g(y)] 
res =  [] 
separable, f= f(x) 
separable, g= g(y) 
the end 


[f(x),g(y)]

In [11]:
sol:integrate(1/S[2],y)=integrate(S[1],x);

'integrate(1/g(y),y) = 'integrate(f(x),x)

# Bibliography

[1] C.P. Viazminsky, On separation of variables, arXiv:math/0210167 (https://arxiv.org/pdf/math/0210167.pdf)

[2] J.A. Cid, A simple method to find out when and ordinary differential equation is separable,  International Journal of Mathematical Education in Science and Technology Vol. 40 , Iss. 5,2009 (http://www4.ujaen.es/~angelcid/Archivos/Papers/IJMEST.pdf)